In [9]:
"""
安裝套件
"""
!pip install line-bot-sdk flask flask-ngrok

In [10]:
"""
將模型的壓縮檔解壓縮
"""

# 引用套件
from zipfile import ZipFile


# 此處是先至teachable-machine網站建立一個模型，下載zip檔案後，再上傳到此colab中
with ZipFile("converted_savedmodel.zip", "r") as zipObj:
   
   # Extract all the contents of zip file in different directory

   zipObj.extractall('converted_savedmodel') # 提取到這個目錄中

In [11]:
"""
1. 載入模型的類別列表，並用for迴圈取得類別的鍵值對

2. 準備一個空類別字典將模型的類別列表寫入



e.g.

key  value
0   蘋果
1   香蕉
2   葡萄

"""

class_dict = {}
with open('converted_savedmodel/labels.txt') as f:
    for line in f:
       (key, val) = line.split()
       class_dict[int(key)] = val

In [12]:
# 引用套件
from flask import Flask, request, abort

from linebot import LineBotApi, WebhookHandler

from linebot.exceptions import InvalidSignatureError

from linebot.models import MessageEvent, TextMessage, TextSendMessage

from flask_ngrok import run_with_ngrok


# 設置主程序 => 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 製作實體物件
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")


# 設置主程序的API接口
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [13]:
"""

當有關注事件時，功能說明 (告訴前台的handler該做什麼)

"""

# 引用套件
from linebot.models import FollowEvent, TextSendMessage

# 關注事件處理
# 當有用戶關注時，回傳設定好的訊息給用戶
@handler.add(FollowEvent)
def process_follow_event(event):
    # 消息發送
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="""這個教室裡面，我置入了兩個業配，當大家找到了業配，今天課程才算開始。請找到那個業配，拍下來並上傳。"""))


In [14]:
"""

當有文字訊息傳來時，功能說明 (告訴前台的handler該做什麼)

"""

# 引用套件
from linebot.models import MessageEvent, TextMessage, TextSendMessage

# 文字消息處理
# 當收到用戶傳來的文字訊息時，回傳已設定好的訊息給用戶
@handler.add(MessageEvent, message=TextMessage)
def process_text_message_event(event):
    # 消息發送
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="""這個教室裡面，我置入了兩個業配，當大家找到了業配，今天課程才算開始。請找到那個業配，拍下來並上傳。"""))

In [15]:
"""

當收到圖片訊息時，功能說明 (告訴前台的handler該做什麼)

在此處，當我們收到圖片訊息時，我們將導入AI模型，然後試著去解析圖片(透過模型判斷是什麼圖片)

流程:
1. 收到用戶傳送之圖片
2. 處理圖片 => 圖片開啟後，尺寸的修改，將圖片轉成矩陣，色塊(圖片矩陣)正規化，重新改寫回原本的圖片資料變數中
3. 藉由既有之模型來進行影像的預測 => 根據不同的機率給出不同的答案

"""
# 引用必須的函式庫及模組
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

from linebot.models import MessageEvent, ImageMessage, TextSendMessage

# 禁用科學記號表達
np.set_printoptions(suppress=True)

# 載入模型 (根據前面模型解壓縮的路徑去做微調)
# tensorflow.keras.models.load_model("模型所在的路徑")
model = load_model("converted_savedmodel/model.savedmodel")


# 引用時間模組
import time

# 當用戶傳入圖像訊息時，告訴handler該如何處理
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    """ 先處理用戶傳來的圖片 """
    
    # 印出當前時間
    print(time.asctime( time.localtime(time.time()) ))

    # 讀取用戶傳來的圖片訊息內容
    message_content = line_bot_api.get_message_content(event.message.id)
    file_name = event.message.id + ".jpg"

    # 打開新圖片然後將用戶傳入之圖片內容寫入到新圖片中
    with open(file_name, "wb") as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

    #######################################################################
    
    """ 再進行圖片的預處理 (pre-processing) """

    # 印出當前時間
    print(time.asctime( time.localtime(time.time()) ))

    # 將用戶的圖片轉換成一個PIL圖片物件，準備用來對圖片進行處理
    image = Image.open(file_name)

    # 將PIL圖片物件重新處理成224*224的圖片
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    # 印出當前時間
    print(time.asctime( time.localtime(time.time()) ))
    
    # 將PIL圖片物件轉換成array物件
    image_array = np.asarray(image)

    # 呈現圖片瞧瞧
    image.show()

    #######################################################################
    
    # 圖片的正規化 (將上方圖片的array物件轉成「浮點數型態」的array物件)
    normalized_image_array = (image_array.astype(np.float32) / 127.0 - 1 )

    # 先建立一個新array(224*224)，用以存放圖片資料
    # 接著將正規化後的圖片array物件放入新建立的array中
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0]= normalized_image_array[0:224,0:224,0:3]

    #######################################################################

    # 用先前載入的模型對用戶傳來的照片進行預測 (圖片需先處理過才能套用AI模型)
    prediction = model.predict(data)

    # 印出當前時間
    print(time.asctime( time.localtime(time.time()) ))

    # 返回預測結果的最大值 (0~1之間)
    max_probability_item_index = np.argmax(prediction[0])

    # 依據預測的結果來做判斷
    if prediction.max() > 0.6:
        line_bot_api.reply_message(event.reply_token, TextSendMessage("""這個物件極有可能是 %s ，其相似機率為 %s """ % (class_dict.get(max_probability_item_index), prediction[0][max_probability_item_index])))
    else :
      line_bot_api.reply_message(event.reply_token, TextSendMessage("""再混啊！亂拍照！！"""))


In [17]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://18f8c22d9bed.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/May/2021 19:07:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 19:08:04] "POST / HTTP/1.1" 200 -


Sat May 22 19:08:22 2021
Sat May 22 19:08:23 2021
Sat May 22 19:08:23 2021
Sat May 22 19:08:24 2021


127.0.0.1 - - [22/May/2021 19:08:24] "POST / HTTP/1.1" 200 -


Sat May 22 19:08:40 2021
Sat May 22 19:08:41 2021
Sat May 22 19:08:41 2021
Sat May 22 19:08:41 2021


127.0.0.1 - - [22/May/2021 19:08:41] "POST / HTTP/1.1" 200 -


Sat May 22 19:09:06 2021
Sat May 22 19:09:07 2021
Sat May 22 19:09:07 2021
Sat May 22 19:09:07 2021


127.0.0.1 - - [22/May/2021 19:09:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2021 19:09:18] "POST / HTTP/1.1" 200 -
